##### Nesta etapa, o objetivo é executar todas as ações necessárias para preparar o DataFrame original para passar pelo processo de modelagem.
        Dessa forma, algumas etapas são necessárias:
                Limpeza e tratamento de dados faltantes e outliers, se necessário;
                Transformação de dados, como a normalização e padronização;
                Feature engineering, com a criação de relações com dados e a verificação de inconsistências;
                Resultado: Um novo DF preparado para a etapa de modelagem. 

### Importando os dados

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# proximos passos: 
# pre processamento - tratamento de nulos, desenvolvimento das ideias das novas variaveis (features) (criá-las postoeriormente permite cross-validation...),
# pipeline do treinamento do modelo -> tratamento de outliers, encoder, normalização, criação propriamente dita da feature
# entrada da base original -> pre processamento -> base de saida preparada para modelagem

In [5]:
train_df = pd.read_csv(r"C:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\Base_dados\train.csv")
df_model = train_df.copy()
df_model.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Executando uma breve verificação se as outras colunas (além de PassengerID, Name e Ticket) são ou não relevantes para a predição de dados:
num_rows = df_model.shape[0]
# Número total de linhas no DataFrame
print(f"O DF tem {num_rows} linhas.")
# Calculando o número de valores únicos em cada coluna
unique_counts = df_model.nunique()
print(unique_counts)

O DF tem 891 linhas.
PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64


• Agora, removeremos as colunas dispensáveis para o nosso modelo, já que não carregam caracteristica preditora, pois tem muitas informações "únicas", além da própria análise do que cada uma delas representa, como por exemplo 'PassengerId', 'Name' ou 'Cabin.

In [7]:
# Executando uma limpeza básica dos dados, excluindo as colunas que não auxiliam na predição:
df_model.drop(columns=["PassengerId", "Name", "Ticket"], inplace= True)

## Data Cleaning

### Tratando os valores nulos

Como visto na etapa do EDA, existiam valores nulos nas colunas de 'Age', 'Cabin' e 'Embarked', assim, preencheremos seus nulos respectivamente com valores impossíveis de serem alcançados por esses parâmetros, respeitando o dtype das colunas.

In [8]:
df_model["Age"] = df_model["Age"].fillna(-1)

In [9]:
# Preenchendo os valores nulos da coluna "Cabin" com dados sintéticos (NA)
df_model["Cabin"] = df_model["Cabin"].fillna('NA')

In [10]:
df_model["Embarked"] = df_model["Embarked"].fillna("NA")
# Verificando as modificações realizadas no data frame original:

In [11]:
# Neste caso há apenas dados nulos em colunas específicas, porém, em nova amostras podem haver dados nulos em outras colunsa, assim, é necessário garantir que o projeto saiba lidar com isso.
# Desta forma, a substituição dos valores nulos será realizada com base no dtype da coluna:

# Criando um dicionário para preencher os valores de acordo com o tipo da coluna:
fill_values = {col: -1 if df_model[col].dtype in ['int64', 'float64'] else 'NA' for col in df_model.columns}

# Usando fillna() com o dicionário
df_model.fillna(value=fill_values, inplace=True)


## Feature Engineering (Teorico)

Introdução de duas novas métricas ao df original que podem auxiliar a previsão do modelo, após segregação de train_test_split, para evitar data leaked.

In [12]:
# Devem ser utilizados os dados da base de treino, para os cálculos, independente de qual base estiver criando a coluna.
# 'Simulando' a criação do train_test_split:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_model, random_state= 15)

A primeira métrica será a Taxa paga pelo passageiro em relação a taxa média paga pela sua classe. 

In [13]:
# Encontrando a Taxa média de cada classe:
taxa_media_classe = df_train.groupby("Pclass")["Fare"].mean()  

# Agora, uma nova coluna deve ser criada: "Taxa_PelaMedia", em ambos os DataSets
df_train["Taxa_PelaMedia"] = df_train["Fare"] / df_train["Pclass"].map(taxa_media_classe)
df_test["Taxa_PelaMedia"] = df_test["Fare"] / df_train["Pclass"].map(taxa_media_classe)

Métrica da Taxa multiplicada a quantidade de irmãos ou conjuges.

In [14]:
df_train["Taxa_x_SibSp"] = df_train["Fare"] * df_train["SibSp"]
df_test["Taxa_x_SibSp"] = df_test["Fare"] * df_test["SibSp"] # Aqui pode-se utilizar os próprio dados pois é um cálculo direto

As duas features anteriores poderiam ser executadas na etapa de modelagem, porém, pela limitação desta base de dados, utilizaremos apenas a segunda, que não é necessário guardar nenhuma outra informação.

Assim, aplicaremos a segunda feature ao df_model:

In [15]:
df_model["Taxa_x_SibSp"] = df_model["Fare"] * df_model["SibSp"]

## Pré Enconding

Como foi visto no EDA, a coluna "Pclass" está com o dtype (int64) errado, como é uma variável categórica, deve ser corrigida:

In [16]:
# Corrigindo o dtype da coluna 'Pclass' para object:
df_model['Pclass'] = df_model['Pclass'].astype(str)

# Verificando a correção realizada
df_model.dtypes 

Survived          int64
Pclass           object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Fare            float64
Cabin            object
Embarked         object
Taxa_x_SibSp    float64
dtype: object

## Salvando os dados

In [17]:
# Utilizando a extensão parquet para salvar os datasets: 
df_model.to_parquet('df_model.parquet')

### Conclusão

Após toda a etapa de pré processamento ser realizada, incluindo a criação do código com as etapas resumidas para aplicação em novos dados, a próxima etapa é a modelagem, onde a execução dos modelos ocorrerá.